<a href="https://colab.research.google.com/github/hwan27/thebest52/blob/master/movie_lens_als.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [3]:
data_path = './data/Books.csv'
data = pd.read_csv(data_path, names=['item', 'user', 'rating', 'timestamp'])

In [4]:
data.head()

,item,user,rating,timestamp
0,0001713353,A1C6M8LCIX4M6M,5.0,1123804800
1,0001713353,A1REUF3A1YCPHM,5.0,1112140800
2,0001713353,A1YRBRK2XM5D5,5.0,1081036800
3,0001713353,A1V8ZR5P78P4ZU,5.0,1077321600
4,0001713353,A2ZB06582NXCIV,5.0,1475452800


In [5]:
data.user.unique()

array(['A1C6M8LCIX4M6M', 'A1REUF3A1YCPHM', 'A1YRBRK2XM5D5', ...,
       'A3P8PUZFHWFK1E', 'AB9M1MQPBAS2J', 'A2TO93KMH3DJIK'], dtype=object)

In [6]:
len(data)

51311621

In [7]:
data.columns[0]

'item'

In [8]:
for col in data.columns:
    print('columns : {}, numbers of unique: {}'.format(col,data[col].nunique()))

columns : item, numbers of unique: 2930451
columns : user, numbers of unique: 15362619
columns : rating, numbers of unique: 6
columns : timestamp, numbers of unique: 7837


In [9]:
item_user_count = data.groupby(['item'])['user'].count()

In [10]:
item_user_count.sort_values(ascending=False)

item
038568231X    58150
0297859382    44956
0007420412    44381
0141353678    37783
0312577222    36620
              ...  
1481933841        1
1481934481        1
1481935429        1
148193564X        1
1443829269        1
Name: user, Length: 2930451, dtype: int64

In [11]:
item_user_count.describe()

count    2.930451e+06
mean     1.750980e+01
std      1.441662e+02
min      1.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      9.000000e+00
max      5.815000e+04
Name: user, dtype: float64

1권당 17회정도 읽힘 최소 1회, 최대 58150회

In [12]:
user_item_count = data.groupby(['user'])['item'].count()

In [13]:
user_item_count.sort_values(ascending=False)

user
A2OJW07GQRNJUT    9684
A2F6N60Z96CAJI    9074
A328S9RN3U5M68    7077
AHUT55E980RDR     5842
A1X8VZWTOG8IS6    4437
                  ... 
A3VNN9KA1UJ4JO       1
A2JVIXLOH0CZI2       1
A2JVIXNUTA7LJU       1
A3VNN9HWNBVM0S       1
AAW5DHZ905LTG        1
Name: item, Length: 15362619, dtype: int64

In [14]:
user_item_count.describe()

count    1.536262e+07
mean     3.340031e+00
std      1.479069e+01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      3.000000e+00
max      9.684000e+03
Name: item, dtype: float64

In [15]:
upper_20_user= user_item_count[user_item_count>20].index

In [16]:
over_20_data = data[data['user'].isin(upper_20_user)]

유저들은 최소 1권이상, 평균 3회정도 책을 리뷰했고 최대 9684권리뷰함

In [17]:
from scipy.sparse import csr_matrix

In [18]:
up_data = over_20_data[over_20_data['rating']>=3]
original_data_size= len(data)
filtered_data_size = len(up_data)

print(f'original_data_size : {original_data_size}, filtered_data_size :{filtered_data_size}')

print(f'Ratio of Remaining Data is {filtered_data_size/original_data_size}')

original_data_size : 51311621, filtered_data_size :14358154
Ratio of Remaining Data is 0.279822654599043


In [19]:
up_data.columns

Index(['item', 'user', 'rating', 'timestamp'], dtype='object')

In [20]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = up_data['user'].unique()
movie_unique = up_data['item'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [21]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = up_data['user'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(up_data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('userId column indexing OK!!')
    up_data['user'] = temp_user_data   # data['userId']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('userId column indexing Fail!!')

# movie_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = up_data['item'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(up_data):
    print('itemId column indexing OK!!')
    up_data['item'] = temp_movie_data
else:
    print('itemId column indexing Fail!!')
    
up_data

userId column indexing OK!!


/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


itemId column indexing OK!!


/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,item,user,rating,timestamp
1,0,0,5.0,1112140800
6,0,1,5.0,1466380800
12,0,2,5.0,1436400000
23,0,3,5.0,1383436800
24,0,4,5.0,1381190400
...,...,...,...,...
51311609,1536996,7477,5.0,1467590400
51311610,1536996,206797,5.0,1467590400
51311612,1536997,134626,4.0,1474848000
51311613,1536997,211094,5.0,1469750400


In [22]:
for col in up_data.columns:
    print('columns : {}, numbers of unique: {}'.format(col,up_data[col].nunique()))

columns : item, numbers of unique: 1536998
columns : user, numbers of unique: 279942
columns : rating, numbers of unique: 3
columns : timestamp, numbers of unique: 7808


In [18]:
up_data.shape

(14358154, 4)

In [39]:
1536998/14358154

0.1070470479700942

In [23]:
item_user_count.describe()

count    2.930451e+06
mean     1.750980e+01
std      1.441662e+02
min      1.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      9.000000e+00
max      5.815000e+04
Name: user, dtype: float64

In [24]:
del data

In [25]:
num_user = up_data['user'].nunique()
num_item = up_data['item'].nunique()

csr_data = csr_matrix((up_data['rating'], (up_data.user, up_data.item)), shape= (num_user, num_item))
# print(csr_data)

In [31]:
# import multiprocessing
# multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=multiprocessing.cpu_count()-4)

# import multiprocessing as mp

# # Step 1: Init multiprocessing.Pool()
# pool = mp.Pool(mp.cpu_count())

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, 
                                    iterations=15, dtype=np.float32,calculate_training_loss=True,
                                   num_threads=-1)

#item x user
csr_data_transpose = csr_data.T
csr_data_transpose

<1536998x279942 sparse matrix of type '<class 'numpy.float64'>'
	with 14247386 stored elements in Compressed Sparse Column format>

In [32]:
als_model.fit(csr_data_transpose)

In [38]:
als_model.item_factors.shape

(1536998, 100)

In [36]:
als_model.calculate_training_loss

True

In [39]:
als_model.user_factors.shape

(279942, 100)

In [44]:
gm_vector = als_model.user_factors[0]

In [46]:
#gm_vector

In [63]:
from sklearn.metrics.pairwise import cosine_similarity as cs
cs(gm_vector.reshape(1,-1),gm_vector.reshape(1,-1))

array([[0.9999999]], dtype=float32)

In [65]:
#knn distance
als_model.similar_users(0)

[(0, 0.030050876),
 (181192, 0.023855792),
 (58814, 0.023755869),
 (276393, 0.023589756),
 (232733, 0.023472203),
 (250061, 0.023316473),
 (237529, 0.02320044),
 (138284, 0.023191405),
 (195841, 0.023189666),
 (66245, 0.023140028)]

### Try Pyspark

In [21]:
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

In [ ]:
num_user = up_data['user'].nunique()
num_item = up_data['item'].nunique()

csr_data = csr_matrix((up_data['rating'], (up_data.user, up_data.item))
                      , shape= (num_user, num_item))
# print(csr_data)
#csr_data_transpose = csr_data.T

#Movie Embeddings
U, sigma, Vt = svds(csr_data, k = 100)
# print(Vt.shape)
V = Vt.transpose()
# print(V.shape)
movie_list = V.tolist()
# movie_embeddings_dict = {columns[i]:tf.convert_to_tensor(movie_list[i]) for i in range(len(columns))}

In [3]:
!pip install pyspark

     |████████████████████████████████| 204.2 MB 1.2 MB/s eta 0:00:01     |█████████████████████████████▎  | 187.1 MB 698 kB/s eta 0:00:25
     |████████████████████████████████| 198 kB 2.1 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612244 sha256=bf8c429ff4fbbdd62c874383b0e74e22a88568d900efe60ad51d52e093fd2858
  Stored in directory: /home/aiffel0042/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark
You should consider upgrading via the '/home/aiffel0042/anaconda3/envs/aiffel/bin/python3 -m pip install --upgrade pip' command.


In [20]:
import multiprocessing
multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count()-1)

import multiprocessing as mp

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

In [ ]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ALSRecommenderSystem').getOrCreate()

DF=spark.createDataFrame(up_data)

mat = RowMatrix(DF)

# Compute the top 5 singular values and corresponding singular vectors.
svd = mat.computeSVD(100, computeU=True)
U = svd.U       # The U factor is a RowMatrix.
# s = svd.s       # The singular values are stored in a local dense vector.
V = svd.V       # The V factor is a local dense matrix.

In [27]:
!pip install findspark

You should consider upgrading via the '/home/aiffel0042/anaconda3/envs/aiffel/bin/python3 -m pip install --upgrade pip' command.


In [28]:
import findspark

In [29]:
findspark.init()

ValueError: Couldn't find Spark, make sure SPARK_HOME env is set or Spark is in an expected location (e.g. from homebrew installation).